# 自定义层
深度学习的⼀个魅⼒在于神经⽹络中各式各样的层，例如全连接层和后⾯章节中将要介绍的卷积
层、池化层与循环层。虽然Gluon提供了⼤量常⽤的层，但有时候我们依然希望⾃定义层。本节
将介绍如何使⽤NDArray来⾃定义⼀个Gluon的层，从而可以被重复调⽤。
## 不含模型参数的⾃定义层
我们先介绍如何定义⼀个不含模型参数的⾃定义层。事实上，这和“模型构造” ⼀节中介绍的使
⽤Block类构造模型类似。下⾯的CenteredLayer类通过继承Block类⾃定义了⼀个将输⼊减
掉均值后输出的层，并将层的计算定义在了forward函数⾥。这个层⾥不含模型参数

In [6]:
from mxnet import gluon, nd
from mxnet.gluon import nn
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

我们可以实例化这个层，然后做前向计算。

In [2]:
net = CenteredLayer()
net(nd.array([1,2,3,4,5,6]))


[-2.5 -1.5 -0.5  0.5  1.5  2.5]
<NDArray 6 @cpu(0)>

我们也可以⽤它来构造更复杂的模型。

In [3]:
net = nn.Sequential()
net.add(nn.Dense(128),CenteredLayer())

下⾯打印⾃定义层各个输出的均值。因为均值是浮点数，所以它的值是⼀个很接近0的数。

In [4]:
net.initialize()
y = net(nd.random.uniform(shape=(4,8)))
y.mean().asscalar()

-7.212293e-10

## 含模型参数的⾃定义层
我们还可以⾃定义含模型参数的⾃定义层。其中的模型参数可以通过训练学出。
[“模型参数的访问、初始化和共享”](4.2parameters.ipynb) ⼀节分别介绍了Parameter类和ParameterDict类。在⾃定
义含模型参数的层时，我们可以利⽤Block类⾃带的ParameterDict类型的成员变量params。
它是⼀个由字符串类型的参数名字映射到Parameter类型的模型参数的字典。我们可以通
过get函数从ParameterDict创建Parameter实例。

In [5]:
params = gluon.ParameterDict()
params.get('params',shape=(2,3))
params

(
  Parameter params (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

现在我们尝试实现⼀个含权重参数和偏差参数的全连接层。它使⽤ReLU函数作为激活函数。其
中in_units和units分别代表输⼊个数和输出个数。

In [8]:
class MyDense(nn.Block):
    # units为该层的输出个数， in_units为该层的输⼊个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense,self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units,units))
        self.bias = self.params.get('bias',shape=(units,))
    def forward(self, x):
        linear = nd.dot(x, self.weight.data() + self.bias.data())
        return nd.relu(linear)

下面我们实例化MyDense类并访问它的模型参数。

In [9]:
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

我们可以直接使⽤⾃定义层做前向计算。

In [10]:
dense.initialize()
dense(nd.random.uniform(shape=(2,5)))


[[0.12833305 0.03455773 0.02007208]
 [0.08881921 0.06478509 0.        ]]
<NDArray 2x3 @cpu(0)>

我们也可以使⽤⾃定义层构造模型。它和Gluon的其他层在使⽤上很类似。

In [11]:
net = nn.Sequential()
net.add(MyDense(8,64),
        MyDense(1,8))
net.initialize()
net(nd.random.uniform(shape=(2,64)))


[[0.03783609]
 [0.03518325]]
<NDArray 2x1 @cpu(0)>

## 小结
- 可以通过Block类⾃定义神经⽹络中的层，从而可以被重复调⽤。